In [ ]:
import requests
import pandas as pd

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
#variable tahun
years = list(range(2022,2021,-1))

In [ ]:
#variable kosong untuk menampung hasil scrapping
all_matches=[]

In [ ]:
standings_url ="https://fbref.com/en/comps/9/Premier-League-Stats"

In [ ]:
for year in years:
  data= requests.get(standings_url)
  soup=BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]
  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if'/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

In [ ]:
team_urls

['https://fbref.com/en/squads/18bb7c10/Arsenal-Stats',
 'https://fbref.com/en/squads/b8fd03ef/Manchester-City-Stats',
 'https://fbref.com/en/squads/19538871/Manchester-United-Stats',
 'https://fbref.com/en/squads/b2b47a98/Newcastle-United-Stats',
 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur-Stats',
 'https://fbref.com/en/squads/fd962109/Fulham-Stats',
 'https://fbref.com/en/squads/d07537b9/Brighton-and-Hove-Albion-Stats',
 'https://fbref.com/en/squads/cd051869/Brentford-Stats',
 'https://fbref.com/en/squads/822bd0ba/Liverpool-Stats',
 'https://fbref.com/en/squads/cff3d9bb/Chelsea-Stats',
 'https://fbref.com/en/squads/8602292d/Aston-Villa-Stats',
 'https://fbref.com/en/squads/47c64c55/Crystal-Palace-Stats',
 'https://fbref.com/en/squads/e4a775cb/Nottingham-Forest-Stats',
 'https://fbref.com/en/squads/a2d435b3/Leicester-City-Stats',
 'https://fbref.com/en/squads/5bfb9659/Leeds-United-Stats',
 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers-Stats',
 'https://fbre

In [ ]:
import time

for year in years:
  data= requests.get(standings_url)
  soup = BeautifulSoup(data.text)
  standings_table = soup.select('table.stats_table')[0]

  links = [l.get("href") for l in standings_table.find_all('a')]
  links = [l for l in links if '/squads/' in l]
  team_urls = [f"https://fbref.com{l}" for l in links]

  for team_url in team_urls:
    team_name = team_url.split("/")[-1].replace("-Stats","").replace("-"," ")

    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]

    soup = BeautifulSoup(data.text)
    links = [l.get("href") for l in soup.find_all('a')]
    links = [l for l in links if l and '/all_comps/shooting' in l]
    links = f"https://fbref.com{links[0]}"
    data1 = requests.get(links)
    shooting = pd.read_html(data1.text, match="Shooting")[0]
    shooting.columns = shooting.columns.droplevel()

    try:
      team_data = matches.merge(shooting[["Date","Sh","SoT","Dist","FK","PK","PKatt"]],on="Date")
    except ValueError:
      continue
    team_data = team_data[team_data["Comp"] == "Premier League"]
    team_data["Season"] = year
    team_data["Team"] = team_name
    all_matches.append(team_data)
    time.sleep(3)

In [ ]:
len(all_matches)

20

In [ ]:
match_df = pd.concat(all_matches)

In [ ]:
match_df.columns = [c.lower() for c in match_df.columns]

In [ ]:
match_df

,date,time,comp,round,day,venue,result,gf,ga,opponent,...,match report,notes,sh,sot,dist,fk,pk,pkatt,season,team
0,2022-08-05,20:00,Premier League,Matchweek 1,Fri,Away,W,2.0,0.0,Crystal Palace,...,Match Report,NaN,10.0,2.0,14.6,1.0,0,0,2022,Arsenal
1,2022-08-13,15:00,Premier League,Matchweek 2,Sat,Home,W,4.0,2.0,Leicester City,...,Match Report,NaN,19.0,7.0,13.0,0.0,0,0,2022,Arsenal
2,2022-08-20,17:30,Premier League,Matchweek 3,Sat,Away,W,3.0,0.0,Bournemouth,...,Match Report,NaN,14.0,6.0,14.8,0.0,0,0,2022,Arsenal
3,2022-08-27,17:30,Premier League,Matchweek 4,Sat,Home,W,2.0,1.0,Fulham,...,Match Report,NaN,22.0,8.0,15.5,1.0,0,0,2022,Arsenal
4,2022-08-31,19:30,Premier League,Matchweek 5,Wed,Home,W,2.0,1.0,Aston Villa,...,Match Report,NaN,22.0,8.0,16.3,1.0,0,0,2022,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2022-11-12,15:00,Premier League,Matchweek 16,Sat,Away,L,1,3,Liverpool,...,Match Report,NaN,9.0,5.0,11.9,0.0,0,0,2022,Southampton
18,2022-12-26,15:00,Premier League,Matchweek 17,Mon,Home,L,1,3,Brighton,...,Match Report,NaN,13.0,3.0,16.1,1.0,0,1,2022,Southampton
19,2022-12-31,15:00,Premier League,Matchweek 18,Sat,Away,L,1,2,Fulham,...,Match Report,NaN,9.0,3.0,24.5,1.0,0,0,2022,Southampton
20,2023-01-04,19:30,Premier League,Matchweek 19,Wed,Home,L,0,1,Nott'ham Forest,...,Match Report,NaN,8.0,0.0,14.5,1.0,0,0,2022,Southampton


In [ ]:
#import menjadi csv file
match_df.to_csv("matches.csv")